## 1.1.1 벡터와 행렬

In [2]:
import numpy as np

In [2]:
x = np.array([1, 2, 3])

In [3]:
x.__class__

numpy.ndarray

In [4]:
x.shape

(3,)

In [5]:
x.ndim

1

In [7]:
W = np.array([[1, 2, 3], [4, 5, 6]])

In [8]:
W.shape

(2, 3)

In [9]:
W.ndim

2

## 1.1.2 행렬의 원소별 연산

In [10]:
W = np.array([[1, 2, 3], [4, 5, 6]])
X = np.array([[0, 1, 2], [3, 4, 5]])

In [11]:
W + X

array([[ 1,  3,  5],
       [ 7,  9, 11]])

In [12]:
W * X

array([[ 0,  2,  6],
       [12, 20, 30]])

## 1.1.3 Broadcast

In [13]:
A = np.array([[1, 2], [3, 4]])

In [14]:
A * 10

array([[10, 20],
       [30, 40]])

In [15]:
A = np.array([[1, 2], [3, 4]])
b = np.array([10, 20])

In [16]:
A * b

array([[10, 40],
       [30, 80]])

## 1.1.4 벡터의 내적과 행렬의 곱

In [18]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

In [19]:
np.dot(a, b)

32

In [20]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

In [21]:
np.matmul(A, B)

array([[19, 22],
       [43, 50]])

## 1.1.5 행렬 형상 확인

In [3]:
W1 = np.random.randn(2, 4)
b1 = np.random.randn(4)
x = np.random.randn(10, 2)

In [4]:
h = np.matmul(x, W1) + b1

In [6]:
h.shape

(10, 4)

In [7]:
def sigmoid(x):
    return 1 / 1 + np.exp(-x)

In [8]:
# 시그모이드 함수로 인해 비선형 변환이 가능
a = sigmoid(h)

In [9]:
x = np.random.randn(10, 2)
W1 = np.random.randn(2, 4)
b1 = np.random.randn(4)
W2 = np.random.randn(4, 3)
b2 = np.random.randn(3)

In [10]:
h = np.matmul(x, W1) + b1
a = sigmoid(h)
s = np.matmul(a, W2) + b2

In [12]:
s.shape

(10, 3)

+ 3차원의 데이터를 출력한다. 따라서 각 차원의 값을 이용하여 3 클래스 분류가 가능.
+ 첫 번째 뉴런이 첫 번째 클래스, 두 번째 뉴런이 두 번쨰 클래스 ...
+ 가장 큰 값을 뱉는 뉴런에 해당하는 클래스가 예측 결과가 되는 것.

## 1.2.2 계층으로 클래스화 및 순전파 구현

In [14]:
class Sigmoid:
    def __init__(self):
        self.params = []
        
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

In [15]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        return out

In [17]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 가중치, 편향 초기화
        W1 = np.random.randn(I, H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H, O)
        b2 = np.random.randn(O)
        
        # 계층 생성
        
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        
        self.params = []
        for layer in self.layers:
            self.params += layer.params
            
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [18]:
a = ['A', 'B']
a += ['C', 'D']

In [19]:
a

['A', 'B', 'C', 'D']

In [20]:
x = np.random.randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)

In [22]:
s.shape

(10, 3)

## 1.3.1 Loss Function

+ 학습이 얼마나 잘 되고 있는지를 확인 시켜주는 것이 바로 'loss'
+ 다중 클래스 분류에는 Cross Entropy Loss 를 사용한다

+ softmax를 통과하면 출력들의 합이 1로 되기 때문에 우리는 이를 'probability'로 해석할 수 있다

In [23]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 정답 데이터가 원핫 벡터일 경우 정답 레이블 인덱스로 변환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]

    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [24]:
# softmax and entropy loss combined
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmax의 출력
        self.t = None  # 정답 레이블

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 정답 레이블이 원핫 벡터일 경우 정답의 인덱스로 변환
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx

## 1.3.2 미분과 기울기

+ 학습의 목표는 손실을 최소화하는 매개변수를 찾는 것이며,
+ 이 과정에서 중요한 것이 '미분'과 '기울기'이다

## 1.3.3 연쇄 법칙

+ 오차역전파법을 이용해 학습을 진행한다. 이때 이용하는 개념이 '연쇄 법칙'
+ '연쇄 법칙' == '합성함수에 대한 미분의 법칙'

+ 이 연쇄 법칙이 중요한 이유는, 우리가 다루는 함수가 아무리 복잡하더라도 그 미분은
+ 별개 함수의 미분들을 이용해 구할 수 있기 때문입니다. 달리 말하면, 각 함수의 국소적인
+ 미분을 계산할 수 있다면 그 값들을 곱해서 전체의 미분을 구할 수 있다는 것.

## 1.3.4 계산 그래프

![1](https://user-images.githubusercontent.com/40786348/57572336-a4f2e480-7453-11e9-8019-fb9bfd7ddca6.PNG)


+ 위는 한 예시
+ chain rule 활용해서 순차적으로 계산해보면 된다

In [34]:
# Repeat node
D, N = 8, 7
x = np.random.randn(1, D)
y = np.repeat(x, N, axis=0)
dy = np.random.rand(N, D)
dx = np.sum(dy, axis=0, keepdims=True)

In [35]:
# Sum node
D, N = 8, 7
x = np.random.randn(N, D)
y = np.sum(x, axis=0, keepdims=True)
dy = np.random.randn(1, D)
dx = np.repeat(dy, N, axis=0)

In [36]:
# Matmul node

class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

# 학습 매개변수를 params에 저장
# 기울기는 grads에 저장

## 1.3.5 기울기 도출과 역전파 구현

In [38]:
class Sigmoid:
    def __init__(self):
        self.params, self.grad = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

## 1.3.5 가중치 갱신

1. 미니배치 - 훈련 데이터 중에서 무작위로 다수의 데이터를 골라낸다.
2. 기울기 계산 - 오차역전파법으로 각 가중치 매개변수에 대한 손실 함수의 기울기를 구한다.
3. 매개변수 갱신 - 기울기를 사용하여 가중치 매개변수를 갱신한다.
4. 반복 - 1~3단계를 필요한 만큼 반복한다.

In [39]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

In [ ]:
model = TwoLayerNet(...)
optimizer = SGD()

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...) # 미니배치 획득
    loss = model.forward(x_batch, t_batch)
    model.backward()
    optimizer.update(model.params, model.grads)
    ...

### 기타 optimizers

In [ ]:
class Momentum:
    '''
    모멘텀 SGG(Momentum SGD)
    '''
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = []
            for param in params:
                self.v.append(np.zeros_like(param))

        for i in range(len(params)):
            self.v[i] = self.momentum * self.v[i] - self.lr * grads[i]
            params[i] += self.v[i]
            
            
class Nesterov:
    '''
    네스테로프 가속 경사(NAG; Nesterov's Accelerated Gradient) (http://arxiv.org/abs/1212.0901)
    '네스테로프 모멘텀 최적화'라고도 한다.
    '''
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = []
            for param in params:
                self.v.append(np.zeros_like(param))

        for i in range(len(params)):
            self.v[i] *= self.momentum
            self.v[i] -= self.lr * grads[i]
            params[i] += self.momentum * self.momentum * self.v[i]
            params[i] -= (1 + self.momentum) * self.lr * grads[i]

In [ ]:
class AdaGrad:
    '''
    AdaGrad
    '''
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = []
            for param in params:
                self.h.append(np.zeros_like(param))

        for i in range(len(params)):
            self.h[i] += grads[i] * grads[i]
            params[i] -= self.lr * grads[i] / (np.sqrt(self.h[i]) + 1e-7)

In [ ]:
class RMSprop:
    '''
    RMSprop
    '''
    def __init__(self, lr=0.01, decay_rate = 0.99):
        self.lr = lr
        self.decay_rate = decay_rate
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = []
            for param in params:
                self.h.append(np.zeros_like(param))

        for i in range(len(params)):
            self.h[i] *= self.decay_rate
            self.h[i] += (1 - self.decay_rate) * grads[i] * grads[i]
            params[i] -= self.lr * grads[i] / (np.sqrt(self.h[i]) + 1e-7)

In [ ]:
class Adam:
    '''
    Adam (http://arxiv.org/abs/1412.6980v8)
    '''
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = [], []
            for param in params:
                self.m.append(np.zeros_like(param))
                self.v.append(np.zeros_like(param))
        
        self.iter += 1
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)

        for i in range(len(params)):
            self.m[i] += (1 - self.beta1) * (grads[i] - self.m[i])
            self.v[i] += (1 - self.beta2) * (grads[i]**2 - self.v[i])
            
            params[i] -= lr_t * self.m[i] / (np.sqrt(self.v[i]) + 1e-7)

## 1.4.2 신경망 구현

In [40]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 가중치 편향 초기화
        W1 = 0.01 * np.random.randn(I, H)
        b1 = np.zeros(H)
        W2 = 0.01 * np.random.randn(H, O)
        b2 = np.zeros(O)
        
        # 계층 생성
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        self.loss_layer = SoftmaxWithLoss()
        
        # 가중치, 기울기 모으기
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def forward(self, x, t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score, t)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

## 1.4.3 학습용 코드 예시

In [ ]:
from ... import SGD
from ... import TwoLayerNet

# 하이퍼파라미터 설정
max_epoch = 300
batch_size = 32
hidden_size = 10
learning_rate = 0.01

x, t = load_data(...)
model = TwoLayerNet(input_size=2, hidden_size=hidden_size, output_size=3)
optimizer = SGD(lr=learning_rate)

data_size = len(x)
max_iters = data_size // batch_size
total_loss = 0
loss_count = 0
loss_list = []

for epoch in range(max_epoch):
    # data shuffle
    idx = np.random.permutation(data_size)
    x = x[idx]
    t = t[idx]
    
    for iters in range(max_iters):
        batch_x = x[iters*batch_size:(iters+1)*batch_size]
        batch_t = t[iters*batch_size:(iters+1)*batch_size]
        
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        
        total_loss += loss
        loss_count += 1
        
        # 학습 결과 출력
        if (iters+1) % 10 == 0:
            avg_loss = total_loss / loss_count
            print("에폭 %d, 반복 %d / %d, 손실 %.2f" % (epoch+1, iters+1, max_iters, avg_loss))
            loss_list.append(avg_loss)
            total_loss, loss_count = 0, 0

## 1.5.1 비트 정밀도

In [42]:
a = np.random.randn(3)
a.dtype

dtype('float64')

+ 넘파이는 64비트 부동소수점 수를 표준으로 사용한다
+ 그러나 신경망의 추론과 학습은 32비트 부동소수점 수로도 문제없이 수행할 수 있다고 한다. (메모리의 효율성)
+ 이러한 이유에서 .astype을 활용하여 데이터 32비트로 바꾸는게 좋다

In [43]:
b = np.random.randn(3).astype(np.float32)
b.dtype

dtype('float32')

In [45]:
c = np.random.randn(3).astype('f')
c.dtype

dtype('float32')

## 1.6 정리

+ 벡터, 행렬부터 시작해서 넘파이 기본적인 사용법을 익혀보았다.
+ 간단한 신경망을 밑바닥부터 구현해보았다. forward, backward 메서드와 params, grads 변수를 갖는다는 '구현 규칙'을 따랐다.
+ 이제 앞으로 이를 바탕으로 자연어에 뛰어들 예정